# 🍸Crawling Set up with Selenium and BeautifulSoup

---

## 😎 기본 세팅

1. 🥂 Colab or Jupyter notebook 을 실행합니다.
    - Jupyter notebook 기준으로 진행하겠습니다.
    - anaconda 설치
        - conda 환경 설정을 원한다면 ```conda create -n [env_name] python=3.9```
        - conda env list 를 통해 가상환경 리스트 조회
        - 원하는 가상환경 활성화 ```conda activate [env_name]```
    - jupyter를 설치합니다 ```conda install jupyter```
2. 🥂 필요 라이브러리 설치 - conda install ~~
    - selenium : crawling tool(Selenium)
    - bs4 : crawling tool(BeautifulSoup)
    - pandas : dataframe library
    - tqdm : process bar
    - requests : request 관련 library
    - lxml : BeautifulSoup 는 기본적으로 HTML 파싱으로 동작하지만 느리다. -> lxml 방식으로 파싱하여 속도 향상


## Selenium
- 셀레니움은 웹드라이버를 통해 페이지를 열고 그 안에서 동적 이벤트를 수행할 수 있습니다. 
- 다만 드라이버를 통해 페이지를 열면서 소요 시간이 길다.
- 동적 크롤링이 필요한 페이지가 아니라면 requests + BeautifulSoup 가 훨씬 빠르다.

1. 🥂 Chrome Driver Download
    - https://chromedriver.chromium.org/downloads
    - Chrome 버전에 맞는 드라이버를 선택합니다.
    - 현재 프로젝트 폴더 내에 chromedriver.exe 를 위치시키거나 저장된 위치를 기억합니다.
    - Selenium 필요시 ```chrome_driver = '저장위치/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)```

## BeautifulSoup
- 페이지 소스를 불러와 원하는 내용들을 뽑아내는 방식
- Selenium에 비해 빠르다


In [1]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
import time as tt
import pandas as pd
from tqdm.notebook import tqdm
import re

#### 현재 작업 환경 확인

In [2]:
os.getcwd()

'C:\\Users\\SSAFY\\Desktop\\Whisky'

### Whisky 전체 리스트 크롤링 
---

- 추출 변수 
    - link : 상세 정보 보기 위한 링크
    - image : 위스키 이미지
    - name : 위스키 이름
    - avr_rating : 평균 평점
    - category : 분류
    - location : 지역

- base_url : https://distiller.com/search?page=
    - 위 url로 검색 시 모든 주류가 검색됩니다.
    - Selenium으로 whisky 버튼 클릭 후 추출하기.
    - 모든 주류 페이지에서 Whisky만 추출하기
    - 모든 주류 목록 페이지 = 3838
    - 위스키 목록 페이지 = 2386
    
✨ 약 1500페이지 차이나는데, Selenium과 BeautifulSoup로 페이지 여는 속도 차이가 약 3~4초 차이 발생
    되도록 동적 이벤트없이 수행하기 위해 BeautifulSoup으로 위스크만 뽑아오겠습니다.
   

In [4]:
chrome_driver = os.getcwd()+"\\chromedriver.exe"
base_url = "https://distiller.com/search?page="
pages = [i for i in range(1,3838)]

In [5]:
url_regex = r"url\(['\"]?([^'\"]*)['\"]?\)" # URL에서 주소를 추출하기 위한 정규 표현식입니다.

In [6]:
## 필요 데이터 exception 처리 method

def text_null_process(content, exp):
    try:
        elem = content.select_one(exp).get_text().strip("\n").strip()
    except AttributeError:
        print("Text NULL 처리")
        elem = None
    return elem

def category_null_process(content, exp):
    try:
        elem = content.select_one(exp).get_text().replace(",", "").strip()
    except AttributeError:
        print("Text NULL 처리")
        elem = None
    return elem



def href_null_process(content, exp):
    try:
        elem = content.select_one(exp)["href"]
    except AttributeError:
        print("Href NULL 처리")
        elem = None
    return elem


def image_null_process(content, exp):
    try:
        elem = re.search(url_regex, content.select_one(exp)["style"]).group(1)
    except AttributeError:
        print("Image NULL 처리")
        elem = None
    return elem


## 크롤링은 할당량

- 다운 : [0:600]
- 창민 : [600:1200]
- 지은 : [1200:1800]
- 은성 : [1800:2400]
- 예진 : [2400:3000]
- 준현 : [3000:]

In [ ]:
links = []
images = []
names = []
avr_ratings = []
categories = []
locations = []
idx = 1
for page_num in tqdm(pages[3000:]):
# for page_num in tqdm([1,2]):
    url = base_url + str(page_num)
    req = requests.get(url)
    tt.sleep(1.5)
    soup = BeautifulSoup(req.text, "lxml")
    whisky_contents = soup.find_all("li", "spirit whiskey-content")
    for content in whisky_contents:
        print(str(page_num) + "페이지 " + str(idx) + " : =====================================================================================================")
        
        link = href_null_process(content, "a")
        image_link = image_null_process(content, "div.image")
        name = text_null_process(content, "div.name")
        average_rating = text_null_process(content, "div.community-rating")
        category = category_null_process(content, "span.accented-text")
        location = text_null_process(content, "span.location")
                
        links.append(link) ## link
        images.append(image_link) ## image link
        names.append(name) ## whisky name
        avr_ratings.append(average_rating) ## average rating
        categories.append(category) ## category
        locations.append(location) ## locatjon
        idx+=1

  0%|          | 0/837 [00:00<?, ?it/s]

3001페이지 1 : =====================================================================================================
3001페이지 2 : =====================================================================================================
3001페이지 3 : =====================================================================================================
3001페이지 4 : =====================================================================================================
3001페이지 5 : =====================================================================================================
3002페이지 6 : =====================================================================================================
3002페이지 7 : =====================================================================================================
3002페이지 8 : =====================================================================================================
3002페이지 9 : ============================================================================

3015페이지 75 : =====================================================================================================
Text NULL 처리
3015페이지 76 : =====================================================================================================
3015페이지 77 : =====================================================================================================
Text NULL 처리
3015페이지 78 : =====================================================================================================
3015페이지 79 : =====================================================================================================
Text NULL 처리
3015페이지 80 : =====================================================================================================
Text NULL 처리
3016페이지 81 : =====================================================================================================
3016페이지 82 : =====================================================================================================
3016페이지 83 : ===============

3027페이지 147 : =====================================================================================================
3027페이지 148 : =====================================================================================================
3027페이지 149 : =====================================================================================================
3027페이지 150 : =====================================================================================================
3028페이지 151 : =====================================================================================================
3028페이지 152 : =====================================================================================================
3028페이지 153 : =====================================================================================================
3028페이지 154 : =====================================================================================================
3028페이지 155 : ==========================================================

3038페이지 217 : =====================================================================================================
3038페이지 218 : =====================================================================================================
3038페이지 219 : =====================================================================================================
3038페이지 220 : =====================================================================================================
3038페이지 221 : =====================================================================================================
3038페이지 222 : =====================================================================================================
3038페이지 223 : =====================================================================================================
3038페이지 224 : =====================================================================================================
3038페이지 225 : ==========================================================

...

3760페이지 4393 : =====================================================================================================
3760페이지 4394 : =====================================================================================================
3760페이지 4395 : =====================================================================================================
3760페이지 4396 : =====================================================================================================
3760페이지 4397 : =====================================================================================================
3760페이지 4398 : =====================================================================================================
3760페이지 4399 : =====================================================================================================
Text NULL 처리
3760페이지 4400 : =====================================================================================================
3760페이지 4401 : ====================================

3768페이지 4464 : =====================================================================================================
3768페이지 4465 : =====================================================================================================
3768페이지 4466 : =====================================================================================================
3768페이지 4467 : =====================================================================================================
3768페이지 4468 : =====================================================================================================
3768페이지 4469 : =====================================================================================================
3768페이지 4470 : =====================================================================================================
3768페이지 4471 : =====================================================================================================
3768페이지 4472 : =================================================

3778페이지 4543 : =====================================================================================================
3778페이지 4544 : =====================================================================================================
3778페이지 4545 : =====================================================================================================
3778페이지 4546 : =====================================================================================================
3778페이지 4547 : =====================================================================================================
3778페이지 4548 : =====================================================================================================
3778페이지 4549 : =====================================================================================================
3779페이지 4550 : =====================================================================================================
3779페이지 4551 : =================================================

3804페이지 4615 : =====================================================================================================
3804페이지 4616 : =====================================================================================================
3804페이지 4617 : =====================================================================================================
3804페이지 4618 : =====================================================================================================
3804페이지 4619 : =====================================================================================================
3804페이지 4620 : =====================================================================================================
3807페이지 4621 : =====================================================================================================
3809페이지 4622 : =====================================================================================================
3812페이지 4623 : =================================================

3820페이지 4687 : =====================================================================================================
3820페이지 4688 : =====================================================================================================
3820페이지 4689 : =====================================================================================================
3820페이지 4690 : =====================================================================================================
3820페이지 4691 : =====================================================================================================
3820페이지 4692 : =====================================================================================================
3820페이지 4693 : =====================================================================================================
3820페이지 4694 : =====================================================================================================
3820페이지 4695 : =================================================

In [ ]:
savePath = os.getcwd()+"\\dataset\\whisky_basic"
savePath

(현재 작업 위치)\\dataset 위치에 생성한 데이터를 csv 파일로 저장합니다.

#### 본인이 크롤링한 시작번호~끝번호로 파일을 저장해주세요

In [ ]:
df = pd.DataFrame({"link": links, "image": images, "name": names, "avr_rating": avr_ratings, "category": categories, "location": locations})
df.to_csv(savePath+"\\save_3000_last.csv", index=True)